In [1]:
suppressMessages(require(Rtsne))
suppressMessages(require(cellrangerRkit))
suppressMessages(require(ggplot2))
suppressMessages(require(data.table))
suppressMessages(require(gplots))
suppressMessages(require(matrixStats))
suppressMessages(require(RColorBrewer))
require(RUVnormalize)

Loading required package: RUVnormalize


In [2]:
venn_matrix=function(inlist){
    #mk matrix
    out_matrix=matrix(NA,length(inlist),length(inlist))
    colnames(out_matrix)=names(inlist)
    rownames(out_matrix)=names(inlist)
    for(i in 2:length(inlist)){
        for(j in 1:(i-1)){
            out_matrix[i,j]=length(intersect(inlist[[i]],inlist[[j]]))
        }
    }
    diag(out_matrix)=sapply(inlist,length)
    return(out_matrix)
}

cross_combining=function(string1,string2){
    o=rep("",length(string1)*length(string2))
    k=0
    for(i in 1:length(string1)){
        for(j in 1:length(string2)){
            k=k+1
            o[k]=paste0(string1[i],"_",string2[j])
        } 
    }
    return(o)
}


In [35]:
genome="GRCh38"
sample_list=c(cross_combining(c("C9","C12"),c("pBMC","R")),cross_combining(c("C10","C17","C18","C22","C23"),c("pBMC","R","I")))

housekeeping_gene=fread("~/Analysis/genomeFiles/housekeeping_gene_human.txt",data.table=F)
housekeeping_gene=housekeeping_gene[,1]

cell_assignment_gps=c("failed BCR/TCR","B cell","T cell","dual-label","unknown")
SC_matrix=list()
ercc_matrix=list()
cell_assignment=list()
sample_gp=c()
BCR_annoation_list=list()
TCR_annoation_list=list()
BCR_annoation_list_weak=list()
TCR_annoation_list_weak=list()

for(i in 1:length(sample_list)){
    #sc
    valid_sc_barcode=load_cellranger_matrix_h5(paste0("/home/ahe/Analysis/201801_JohnVDJ/data/10x/",sample_list[i],"_10x"), genome=genome,barcode_filtered =T)
    valid_sc_barcode=valid_sc_barcode$barcode
    SC_matrix[[i]]=load_cellranger_matrix_h5(paste0("/home/ahe/Analysis/201801_JohnVDJ/data/10x/",sample_list[i],"_10x"), genome=genome,barcode_filtered =F)
    SC_matrix[[i]]=exprs(SC_matrix[[i]])
    #vdj
    BCR_annoation_list[[i]]=fread(paste0("/home/ahe/Analysis/201801_JohnVDJ/data/VDJ/",sample_list[i],"_BCR/outs/all_contig_annotations.csv")
                                  ,sep=",",data.table=F,na.strings = "None")
    TCR_annoation_list[[i]]=fread(paste0("/home/ahe/Analysis/201801_JohnVDJ/data/VDJ/",sample_list[i],"_TCR/outs/all_contig_annotations.csv")
                                  ,sep=",",data.table=F,na.strings = "None")   
    #filter the error signal
    TCRinBCRtbl=grep("TR",BCR_annoation_list[[i]]$chain)
    BCRinTCRtbl=grep("IG",TCR_annoation_list[[i]]$chain)
    if(length(TCRinBCRtbl)>0){BCR_annoation_list[[i]]=BCR_annoation_list[[i]][-TCRinBCRtbl,]}
    if(length(BCRinTCRtbl)>0){TCR_annoation_list[[i]]=TCR_annoation_list[[i]][-BCRinTCRtbl,]}
    
    #continue BCR TCR processing
    valid_BCR_list=which(BCR_annoation_list[[i]]$is_cell & BCR_annoation_list[[i]]$high_confidence & BCR_annoation_list[[i]]$productive)
    valid_TCR_list=which(TCR_annoation_list[[i]]$is_cell & TCR_annoation_list[[i]]$high_confidence & TCR_annoation_list[[i]]$productive)
    valid_B_list=unique(BCR_annoation_list[[i]]$barcode[valid_BCR_list])
    valid_T_list=unique(TCR_annoation_list[[i]]$barcode[valid_TCR_list])
    tag_B=BCR_annoation_list[[i]]$barcode %in% valid_B_list
    tag_T=TCR_annoation_list[[i]]$barcode %in% valid_T_list
    BCR_annoation_list_weak[[i]]=BCR_annoation_list[[i]][!tag_B,]
    TCR_annoation_list_weak[[i]]=TCR_annoation_list[[i]][!tag_T,]
    BCR_annoation_list_weak[[i]]=BCR_annoation_list_weak[[i]][BCR_annoation_list_weak[[i]]$is_cell & BCR_annoation_list_weak[[i]]$high_confidence,]
    TCR_annoation_list_weak[[i]]=TCR_annoation_list_weak[[i]][TCR_annoation_list_weak[[i]]$is_cell & TCR_annoation_list_weak[[i]]$high_confidence,]
    BCR_annoation_list[[i]]=BCR_annoation_list[[i]][tag_B,]
    TCR_annoation_list[[i]]=TCR_annoation_list[[i]][tag_T,]

    #the cell selection
    valid_cell_barcodes=unique(c(valid_sc_barcode,valid_B_list,valid_T_list))
    SC_matrix[[i]]=SC_matrix[[i]][,colnames(SC_matrix[[i]]) %in% valid_cell_barcodes]
    BCR_annoation_list[[i]]=BCR_annoation_list[[i]][BCR_annoation_list[[i]]$barcode %in% valid_cell_barcodes,]
    TCR_annoation_list[[i]]=TCR_annoation_list[[i]][TCR_annoation_list[[i]]$barcode %in% valid_cell_barcodes,]
    #continue SC MATRIX PROCESSION
    SC_matrix[[i]]=t(as.matrix(SC_matrix[[i]]))
    SC_matrix[[i]]=SC_matrix[[i]][rowSums(SC_matrix[[i]]>1)>50,]
    #ercc
    #ercc_matrix[[i]]=load_cellranger_matrix_h5(paste0("/home/ahe/Analysis/201801_JohnVDJ/data/ercc/",sample_list[i],"_10x_ercc"), genome="ercc92",barcode_filtered =F)
    #ercc_matrix[[i]]=exprs(ercc_matrix[[i]])
    #ercc_matrix[[i]]=ercc_matrix[[i]][,match(rownames(SC_matrix[[i]]),colnames(ercc_matrix[[i]]))]
    #ercc_matrix[[i]]=t(as.matrix(ercc_matrix[[i]]))
    #SC_matrix[[i]]=cbind(ercc_matrix[[i]],SC_matrix[[i]])
    
    #renaming
    rownames(SC_matrix[[i]])=gsub("1$", i, rownames(SC_matrix[[i]]))
    TCR_annoation_list[[i]]$barcode=gsub("1$", i, TCR_annoation_list[[i]]$barcode)
    BCR_annoation_list[[i]]$barcode=gsub("1$", i, BCR_annoation_list[[i]]$barcode)
    TCR_annoation_list_weak[[i]]$barcode=gsub("1$", i, TCR_annoation_list_weak[[i]]$barcode)
    BCR_annoation_list_weak[[i]]$barcode=gsub("1$", i, BCR_annoation_list_weak[[i]]$barcode)   
    
    #cell labeling
    cell_assignment[[i]]=rep("unknown",nrow(SC_matrix[[i]]))
    cell_assignment[[i]][rownames(SC_matrix[[i]]) %in% BCR_annoation_list_weak[[i]]$barcode |
                         rownames(SC_matrix[[i]]) %in% TCR_annoation_list_weak[[i]]$barcode]="failed BCR/TCR"
    cell_assignment[[i]][rownames(SC_matrix[[i]]) %in% BCR_annoation_list[[i]]$barcode]="B cell"
    cell_assignment[[i]][rownames(SC_matrix[[i]]) %in% TCR_annoation_list[[i]]$barcode]="T cell"
    cell_assignment[[i]][rownames(SC_matrix[[i]]) %in% TCR_annoation_list[[i]]$barcode & 
                         rownames(SC_matrix[[i]]) %in% BCR_annoation_list[[i]]$barcode]="dual-label"
    
    #remove the redundant unknown cells
    #if(sum(cell_assignment[[i]]=="unknown")>num_of_unknown_cell_at_most_per_lirary){
        #tag_unknown=which(cell_assignment[[i]]=="unknown")
        #discard_unknown=sample(tag_unknown,length(tag_unknown)-num_of_unknown_cell_at_most_per_lirary,replace = F)
        #SC_matrix[[i]]=SC_matrix[[i]][-discard_unknown,]
        #cell_assignment[[i]]=cell_assignment[[i]][-discard_unknown]
    #}    

    sample_gp=c(sample_gp,rep(i,nrow(SC_matrix[[i]])))
}

ExpressionMat=do.call(rbind,SC_matrix)
ExpressionMat=ExpressionMat[,colSums(ExpressionMat)>2]
ExpressionBinaryMat=ExpressionMat>=1
ExpressionMat=ExpressionMat[,colSums(ExpressionBinaryMat)>nrow(ExpressionMat)/100]
#ExpressionMat=t(apply(ExpressionMat,1,function(x){x/sum(x)*10000000}))

mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
  dataset = "hsapiens_gene_ensembl",  #human
  host = 'ensembl.org')
#biomaRt::listAttributes(mart)
t2g <- biomaRt::getBM(attributes = c("ensembl_transcript_id", "ensembl_gene_id",
    "external_gene_name","chromosome_name","start_position","end_position","strand"), mart = mart)
t2g <- dplyr::rename(t2g, target_id = ensembl_transcript_id,
  ens_gene = ensembl_gene_id, ext_gene = external_gene_name)

#change name
ExpressionMat=ExpressionMat[,which(colnames(ExpressionMat) %in% t2g$ens_gene)]
colnames(ExpressionMat)=t2g$ext_gene[match(colnames(ExpressionMat),t2g$ens_gene)]
colnames(ExpressionMat)[which(duplicated(colnames(ExpressionMat)))]=paste0(colnames(ExpressionMat)[which(duplicated(colnames(ExpressionMat)))],"_1")

cell_assignment=unlist(cell_assignment)
BCR_annoation=do.call(rbind,BCR_annoation_list)
TCR_annoation=do.call(rbind,TCR_annoation_list)
BCR_annoation_w=do.call(rbind,BCR_annoation_list_weak)
TCR_annoation_w=do.call(rbind,TCR_annoation_list_weak)
#rm(SC_matrix,BCR_annoation_list,TCR_annoation_list,TCR_annoation_list_weak,BCR_annoation_list_weak)

## QC: Venn

In [38]:
#check BCR TCR
#require(repr)
#options(repr.plot.width=8, repr.plot.height=4)
pdf("cells label overlaping between TCR_BCR samples_including weak.pdf",width=14,height=7)
par(mfrow=c(1,2))
venn(list(failed_TCR=unique(TCR_annoation_w$barcode),
          TCR=unique(TCR_annoation$barcode),
          BCR=unique(BCR_annoation$barcode),
          failed_BCR=unique(BCR_annoation_w$barcode)
         ),small=0.5)
title(main="all")
plot(1, type="n", axes=F, xlab="", ylab="")
for(i in 1:length(sample_list)){
    tagSC=grep(paste0("-",i,"$"),rownames(ExpressionMat))
    tagBCR=grep(paste0("-",i,"$"),BCR_annoation$barcode)
    tagTCR=grep(paste0("-",i,"$"),TCR_annoation$barcode)
    tagBCR_w=grep(paste0("-",i,"$"),BCR_annoation_w$barcode)
    tagTCR_w=grep(paste0("-",i,"$"),TCR_annoation_w$barcode)    
    venn(list(failed_TCR=unique(TCR_annoation_w$barcode[tagTCR_w]),
          TCR=unique(TCR_annoation$barcode[tagTCR]),
          BCR=unique(BCR_annoation$barcode[tagBCR]),
          failed_BCR=unique(BCR_annoation_w$barcode[tagBCR_w])),small=0.5)
    title(main=sample_list[i])
}
dev.off()

png 
  2

In [37]:
#check BCR TCR
#require(repr)
#options(repr.plot.width=8, repr.plot.height=4)
pdf("cells label overlaping between SC_TCR_BCR samples.pdf",width=10,height=5)
par(mfrow=c(1,2))
venn(list(SC=rownames(ExpressionMat),
              TCR=TCR_annoation$barcode,
              BCR=BCR_annoation$barcode),small=0.5)
title(main="all")
plot(1, type="n", axes=F, xlab="", ylab="")
for(i in 1:length(sample_list)){
    tagSC=grep(paste0("-",i,"$"),rownames(ExpressionMat))
    tagBCR=grep(paste0("-",i,"$"),BCR_annoation$barcode)
    tagTCR=grep(paste0("-",i,"$"),TCR_annoation$barcode)
    venn(list(SC=rownames(ExpressionMat)[tagSC],
              TCR=unique(TCR_annoation$barcode[tagTCR]),
              BCR=unique(BCR_annoation$barcode[tagBCR])),small=0.5)
    title(main=sample_list[i])
}
dev.off()

png 
  2

In [40]:
recombined_BCR_list=which(is.na(BCR_annoation_w$v_gene)+is.na(BCR_annoation_w$d_gene)+is.na(BCR_annoation_w$j_gene)+is.na(BCR_annoation_w$c_gene)<3)
recombined_TCR_list=which(is.na(TCR_annoation_w$v_gene)+is.na(TCR_annoation_w$d_gene)+is.na(TCR_annoation_w$j_gene)+is.na(TCR_annoation_w$c_gene)<3)
BCR_annoation_w=BCR_annoation_w[recombined_BCR_list,]
TCR_annoation_w=TCR_annoation_w[recombined_TCR_list,]

pdf("cells label overlaping between TCR_BCR samples_with recombination signature.pdf",width=7,height=7)
venn(list(failed_TCR=unique(TCR_annoation_w$barcode),
          TCR=unique(TCR_annoation$barcode),
          BCR=unique(BCR_annoation$barcode),
          failed_BCR=unique(BCR_annoation_w$barcode)
         ),small=0.5)
title(main="all")
dev.off()

png 
  2

## QC: ERCC, MT

In [28]:
#check SD/MEAN ratio
taggene_mt=c(t2g$ext_gene[t2g$chromosome_name=="MT"])
taggene_hk=housekeeping_gene
#require(repr)
#options(repr.plot.width=8, repr.plot.height=4)
pdf("Sd_Mean ratio distribution.pdf",width=8,height=4)
plot(density(colSds(ExpressionMat)/colMeans(ExpressionMat)),xlim=c(0,20),ylim=c(0,3),xlab="Sd/Mean")
lines(density(colSds(ExpressionMat[,colnames(ExpressionMat) %in% taggene_mt])/colMeans(ExpressionMat[,colnames(ExpressionMat) %in% taggene_mt])),col="red")
lines(density(colSds(ExpressionMat[,colnames(ExpressionMat) %in% taggene_hk])/colMeans(ExpressionMat[,colnames(ExpressionMat) %in% taggene_hk])),col="blue")
legend("topright",c("Mitochondria gene","House Keeping gene","all"),lty=1,col=c("red","blue","black"))
dev.off()

png 
  2

In [13]:
#check ERCC
#require(repr)
#options(repr.plot.width=8, repr.plot.height=4)
pdf("ERCC QC UMIs.pdf",width=10,height=5)
coltouse=cell_assignment
tagcol=RColorBrewer::brewer.pal(length(cell_assignment_gps),"Set1")
for (i in 1:length(cell_assignment_gps)){
    coltouse[coltouse==cell_assignment_gps[i]]=tagcol[i]
}

par(mfrow=c(1,2))
plot(density(rowSums(ExpressionMat[,grep("ERCC",colnames(ExpressionMat))])),xlab="ERCC UMI number per cell",ylab="density",main="all")
plot(rowSums(ExpressionMat[,grep("ERCC",colnames(ExpressionMat))]),rowSums(ExpressionMat[,-grep("ERCC",colnames(ExpressionMat))]),
     xlab="ERCC UMI number per cell",ylab="UMI number per cell",pch=19,cex=0.5,col=coltouse)
for(i in 1:length(sample_list)){
    tag=which(sample_gp==i)
    plot(density(rowSums(ExpressionMat[tag,grep("ERCC",colnames(ExpressionMat))])),xlab="ERCC UMI number per cell",ylab="density",main=sample_list[i])
    plot(rowSums(ExpressionMat[tag,grep("ERCC",colnames(ExpressionMat))]),rowSums(ExpressionMat[tag,-grep("ERCC",colnames(ExpressionMat))]),
         xlab="ERCC UMI number per cell",ylab="UMI number per cell",pch=19,cex=0.5,col=coltouse[tag])
}
dev.off()

[1] 703

In [ ]:
#options(repr.plot.width=8, repr.plot.height=4)
pdf("ERCC QC TPMs.pdf",width=10,height=5)
ExpressionFiltered=t(apply(ExpressionMat,1,function(x){x/sum(x)*10000000}))
par(mfrow=c(1,2))
plot(density(rowSums(ExpressionFiltered[,grep("ERCC",colnames(ExpressionFiltered))])),xlab="ERCC UMI number per cell",ylab="density",main="all")
plot(rowSums(ExpressionFiltered[,grep("ERCC",colnames(ExpressionFiltered))]),rowSums(ExpressionFiltered[,-grep("ERCC",colnames(ExpressionFiltered))]),
     xlab="ERCC UMI number per cell",ylab="UMI number per cell",pch=19,cex=0.5,col=coltouse)
for(i in 1:length(sample_list)){
    tag=which(sample_gp==i)
    plot(density(rowSums(ExpressionFiltered[tag,grep("ERCC",colnames(ExpressionFiltered))])),xlab="ERCC UMI number per cell",ylab="density",main=sample_list[i])
    plot(rowSums(ExpressionFiltered[tag,grep("ERCC",colnames(ExpressionFiltered))]),rowSums(ExpressionFiltered[tag,-grep("ERCC",colnames(ExpressionFiltered))]),
         xlab="ERCC UMI number per cell",ylab="UMI number per cell",pch=19,cex=0.5,col=coltouse[tag])
}
dev.off()

In [ ]:
#options(repr.plot.width=8, repr.plot.height=4)
pdf("ERCC QC RUV.pdf",width=10,height=5)
ExpressionNormed=scale(ExpressionFiltered,scale=F)
ExpressionNormed=naiveRandRUV(ExpressionNormed, grep("ERCC",colnames(ExpressionNormed)), nu.coeff=1e-3, k=20)
ExpressionNormed=scale(ExpressionNormed[,-grep("ERCC",colnames(ExpressionNormed))])
par(mfrow=c(1,2))
plot(density(rowSums(ExpressionNormed[,grep("ERCC",colnames(ExpressionNormed))])),xlab="ERCC UMI number per cell",ylab="density",main="all")
plot(rowSums(ExpressionNormed[,grep("ERCC",colnames(ExpressionNormed))]),rowSums(ExpressionNormed[,-grep("ERCC",colnames(ExpressionNormed))]),
     xlab="ERCC UMI number per cell",ylab="UMI number per cell",pch=19,cex=0.5,col=coltouse)
for(i in 1:length(sample_list)){
    tag=which(sample_gp==i)
    plot(density(rowSums(ExpressionNormed[tag,grep("ERCC",colnames(ExpressionNormed))])),xlab="ERCC UMI number per cell",ylab="density",main=sample_list[i])
    plot(rowSums(ExpressionNormed[tag,grep("ERCC",colnames(ExpressionNormed))]),rowSums(ExpressionNormed[tag,-grep("ERCC",colnames(ExpressionNormed))]),
         xlab="ERCC UMI number per cell",ylab="UMI number per cell",pch=19,cex=0.5,col=coltouse[tag])
}
dev.off()

# other QC

In [29]:
sample_list

[1] "C9_pBMC"  "C9_R"     "C12_pBMC" "C12_R"    "C17_pBMC" "C17_R"   
 [7] "C17_I"    "C18_pBMC" "C18_R"    "C18_I"    "C22_pBMC" "C22_R"   
[13] "C22_I"    "C23_pBMC" "C23_R"    "C23_I"

In [30]:
i=16
sample_list[i]
tagBCR_anno=BCR_annoation[grep(paste0("-",i),BCR_annoation$barcode),]

[1] "C23_I"

In [31]:
nrow(tagBCR_anno)

[1] 233854

In [33]:
length(grep("TR",BCR_annoation$chain))

[1] 0

In [35]:
length(unique(tagBCR_anno$barcode))

[1] 50432